# CaseStudy I
In this notebook, real-life MCDA problem will be solved using multiple modules. CaseStudy I refers to selecting a European country in which an electric plant will be built. For 6 country (alternatives) we got 3 criteria: the power (g1), safety level (g2), construction cost (g3). In this case, Decision Maker wants 2 outputs - ranking (Promethee II) and choice (Promethee V)

## Definition of inputs and problem formalization

In [12]:
import pandas as pd
from core.enums import Direction, SurrogateMethod, GeneralCriterion, FlowType
from core.constraint import Constraint, Relation
from modular_parts.weights import surrogate_weights
from modular_parts.preference import compute_reinforced_preference,\
compute_veto
from modular_parts.flows import calculate_promethee_outranking_flows, \
calculate_net_outranking_flows
from modular_parts.ranking import calculate_promethee_ii_ranking
from modular_parts.choice import compute_decision


alternatives = ['ITA', 'BEL', 'GER', 'SWE', 'AUT', 'FRA']
criteria = ['g1', 'g2', 'g3']
criteria_ranks = pd.Series([2, 3, 1], criteria)

generalised_criteria = pd.Series([GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE],
                                 criteria)
criteria_directions = pd.Series([Direction.MAX, Direction.MAX, Direction.MIN],
                                criteria)
indifference_thresholds = pd.Series([0, 0, 100], criteria)
preference_thresholds = pd.Series([0, 1, 300], criteria)
reinforced_preference_thresholds = pd.Series([10, 5, 500], criteria)
reinforcement_factors = pd.Series([1.3, 1.2, 1.5], criteria)
s_parameters = pd.Series([None, None, None], criteria)
veto_thresholds = pd.Series([None, 5, 400], criteria)

alternatives_performances = pd.DataFrame(data=[[98, 8, 400],
                                               [58, 0, 800],
                                               [66, 5, 1000],
                                               [74, 3, 600],
                                               [80, 7, 200],
                                               [82, 10, 600]],
                                         columns=criteria,
                                         index=alternatives)

## Modules usage

Weights calculated with Rank Ordered Centroid method:

In [13]:
weights = surrogate_weights(criteria_ranks, SurrogateMethod.ROC)
weights

g1    0.278
g2    0.111
g3    0.611
Name: weights, dtype: float64

g1    0.278
g2    0.111
g3    0.611
Name: weights, dtype: float64

Preferences are calculated using Promethee Reinforced Preference


In [14]:
preference_indices, partial_preference_indices = compute_reinforced_preference(alternatives_performances,
                                                preference_thresholds,
                                                indifference_thresholds,
                                                s_parameters,
                                                generalised_criteria,
                                                criteria_directions,
                                                reinforced_preference_thresholds,
                                                reinforcement_factors,
                                                weights)
partial_preference_indices

ITA  BEL  GER  SWE  AUT  FRA
criteria                                  
g1       ITA  0.0  1.3  1.3  1.3  1.3  1.3
         BEL  0.0  0.0  0.0  0.0  0.0  0.0
         GER  0.0  1.0  0.0  0.0  0.0  0.0
         SWE  0.0  1.3  1.0  0.0  0.0  0.0
         AUT  0.0  1.3  1.3  1.0  0.0  0.0
         FRA  0.0  1.3  1.3  1.0  1.0  0.0
g2       ITA  0.0  1.2  1.0  1.0  1.0  0.0
         BEL  0.0  0.0  0.0  0.0  0.0  0.0
         GER  0.0  1.0  0.0  1.0  0.0  0.0
         SWE  0.0  1.0  0.0  0.0  0.0  0.0
         AUT  0.0  1.2  1.0  1.0  0.0  0.0
         FRA  1.0  1.2  1.0  1.2  1.0  0.0
g3       ITA  0.0  1.0  1.5  0.5  0.0  0.5
         BEL  0.0  0.0  0.5  0.0  0.0  0.0
         GER  0.0  0.0  0.0  0.0  0.0  0.0
         SWE  0.0  0.5  1.0  0.0  0.0  0.0
         AUT  0.5  1.5  1.5  1.0  0.0  1.0
         FRA  0.0  0.5  1.0  0.0  0.0  0.0

ITA  BEL  GER  SWE  AUT  FRA
criteria                                  
g1       ITA  0.0  1.3  1.3  1.3  1.3  1.3
         BEL  0.0  0.0  0.0  0.0  0.0  0.0
         GER  0.0  1.0  0.0  0.0  0.0  0.0
         SWE  0.0  1.3  1.0  0.0  0.0  0.0
         AUT  0.0  1.3  1.3  1.0  0.0  0.0
         FRA  0.0  1.3  1.3  1.0  1.0  0.0
g2       ITA  0.0  1.2  1.0  1.0  1.0  0.0
         BEL  0.0  0.0  0.0  0.0  0.0  0.0
         GER  0.0  1.0  0.0  1.0  0.0  0.0
         SWE  0.0  1.0  0.0  0.0  0.0  0.0
         AUT  0.0  1.2  1.0  1.0  0.0  0.0
         FRA  1.0  1.2  1.0  1.2  1.0  0.0
g3       ITA  0.0  1.0  1.5  0.5  0.0  0.5
         BEL  0.0  0.0  0.5  0.0  0.0  0.0
         GER  0.0  0.0  0.0  0.0  0.0  0.0
         SWE  0.0  0.5  1.0  0.0  0.0  0.0
         AUT  0.5  1.5  1.5  1.0  0.0  1.0
         FRA  0.0  0.5  1.0  0.0  0.0  0.0

In [15]:
preference_indices

,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.000,1.000,1.000,0.718,0.436,0.616
BEL,0.000,0.000,0.306,0.000,0.000,0.000
GER,0.000,0.389,0.000,0.111,0.000,0.000
SWE,0.000,0.718,0.889,0.000,0.000,0.000
AUT,0.306,1.000,1.000,1.000,0.000,0.611
FRA,0.111,0.724,1.000,0.402,0.389,0.000


,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.000,1.000,1.000,0.718,0.436,0.616
BEL,0.000,0.000,0.306,0.000,0.000,0.000
GER,0.000,0.389,0.000,0.111,0.000,0.000
SWE,0.000,0.718,0.889,0.000,0.000,0.000
AUT,0.306,1.000,1.000,1.000,0.000,0.611
FRA,0.111,0.724,1.000,0.402,0.389,0.000


Next, we filter out the alternatives with an unacceptably weak performance on some criteria using Promethee Veto.

In [16]:
veto_indices, partial_veto_indices  = compute_veto(alternatives_performances, weights,
                                  veto_thresholds, criteria_directions)
partial_veto_indices

ITA  BEL  GER  SWE  AUT  FRA
criteria                                  
g1       ITA    0    0    0    0    0    0
         BEL    0    0    0    0    0    0
         GER    0    0    0    0    0    0
         SWE    0    0    0    0    0    0
         AUT    0    0    0    0    0    0
         FRA    0    0    0    0    0    0
g2       ITA    0    0    0    0    0    0
         BEL    1    0    1    0    1    1
         GER    0    0    0    0    0    1
         SWE    1    0    0    0    0    1
         AUT    0    0    0    0    0    0
         FRA    0    0    0    0    0    0
g3       ITA    0    0    0    0    0    0
         BEL    1    0    0    0    1    0
         GER    1    0    0    1    1    1
         SWE    0    0    0    0    1    0
         AUT    0    0    0    0    0    0
         FRA    0    0    0    0    1    0

ITA  BEL  GER  SWE  AUT  FRA
criteria                                  
g1       ITA    0    0    0    0    0    0
         BEL    0    0    0    0    0    0
         GER    0    0    0    0    0    0
         SWE    0    0    0    0    0    0
         AUT    0    0    0    0    0    0
         FRA    0    0    0    0    0    0
g2       ITA    0    0    0    0    0    0
         BEL    1    0    1    0    1    1
         GER    0    0    0    0    0    1
         SWE    1    0    0    0    0    1
         AUT    0    0    0    0    0    0
         FRA    0    0    0    0    0    0
g3       ITA    0    0    0    0    0    0
         BEL    1    0    0    0    1    0
         GER    1    0    0    1    1    1
         SWE    0    0    0    0    1    0
         AUT    0    0    0    0    0    0
         FRA    0    0    0    0    1    0

In [17]:
veto_indices

,ITA,BEL,GER,SWE,AUT,FRA
ITA,0,0,0,0,0,0
BEL,1,0,1,0,1,1
GER,1,0,0,1,1,1
SWE,1,0,0,0,1,1
AUT,0,0,0,0,0,0
FRA,0,0,0,0,1,0


,ITA,BEL,GER,SWE,AUT,FRA
ITA,0,0,0,0,0,0
BEL,1,0,1,0,1,1
GER,1,0,0,1,1,1
SWE,1,0,0,0,1,1
AUT,0,0,0,0,0,0
FRA,0,0,0,0,1,0


Here, preference indices are computed with veto to achieve overall preferences.

In [18]:
_, _, overall_preference = compute_veto(alternatives_performances, weights,
                                  veto_thresholds, criteria_directions,
                                  preferences=preference_indices)
overall_preference

,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.000,1.000,1.000,0.718,0.436,0.616
BEL,0.000,0.000,0.000,0.000,0.000,0.000
GER,0.000,0.389,0.000,0.000,0.000,0.000
SWE,0.000,0.718,0.889,0.000,0.000,0.000
AUT,0.306,1.000,1.000,1.000,0.000,0.611
FRA,0.111,0.724,1.000,0.402,0.000,0.000


,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.000,1.000,1.000,0.718,0.436,0.616
BEL,0.000,0.000,0.000,0.000,0.000,0.000
GER,0.000,0.389,0.000,0.000,0.000,0.000
SWE,0.000,0.718,0.889,0.000,0.000,0.000
AUT,0.306,1.000,1.000,1.000,0.000,0.611
FRA,0.111,0.724,1.000,0.402,0.000,0.000


Last step before ranking/choice are promethee flows. First outranking flows need to be calculated.

In [19]:
outranking_flows = calculate_promethee_outranking_flows(overall_preference,
                                                        FlowType.BASIC)
outranking_flows

,positive,negative
ITA,0.7540,0.0834
BEL,0.0000,0.7662
GER,0.0778,0.7778
SWE,0.3214,0.4240
AUT,0.7834,0.0872
FRA,0.4474,0.2454


,positive,negative
ITA,0.7540,0.0834
BEL,0.0000,0.7662
GER,0.0778,0.7778
SWE,0.3214,0.4240
AUT,0.7834,0.0872
FRA,0.4474,0.2454


Then, flows are inputs for NetOutrankingFlow (M9) module in order to compute net outranking flow.

In [20]:
prometheeII_flows = calculate_net_outranking_flows(outranking_flows, True)
prometheeII_flows

,positive,negative,net
ITA,0.7540,0.0834,0.6706
BEL,0.0000,0.7662,-0.7662
GER,0.0778,0.7778,-0.7000
SWE,0.3214,0.4240,-0.1026
AUT,0.7834,0.0872,0.6962
FRA,0.4474,0.2454,0.2020


,positive,negative,net
ITA,0.7540,0.0834,0.6706
BEL,0.0000,0.7662,-0.7662
GER,0.0778,0.7778,-0.7000
SWE,0.3214,0.4240,-0.1026
AUT,0.7834,0.0872,0.6962
FRA,0.4474,0.2454,0.2020


Finally, we obtained the ranking and the chosen pair of alternatives.

In [21]:
calculate_promethee_ii_ranking(prometheeII_flows)

1    AUT
2    ITA
3    FRA
4    SWE
5    GER
6    BEL
Name: ranking, dtype: object

1    AUT
2    ITA
3    FRA
4    SWE
5    GER
6    BEL
Name: ranking, dtype: object

In [22]:
constraint1 = Constraint([22, 17, 25, 28, 20, 18], Relation.LEQ, 40)
constraint2 = Constraint([1, 1, 1, 1, 1, 1], Relation.LEQ, 2)
compute_decision(prometheeII_flows['net'], [constraint1, constraint2])

0    AUT
1    FRA
Name: chosen alternatives, dtype: object

0    AUT
1    FRA
Name: chosen alternatives, dtype: object